## PyTorch Linear Regression using built-ins

In [30]:
# Import NumPy, PyTorch, torch.nn
import torch
import numpy as np
import torch.nn as nn

## Problem Statement

The model will predict crop yields for 2 different fruits: apples, oranges.
The input variables are average temperature, rainfall, humidity.

In [31]:
# Input data (temperature, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], 
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], 
                   [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

Before building a model, *input* and *target* need to be converted to PyTorch tensors.

In [32]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Dataset, DataLoader

We'll create a **TensorDataset**, which allows to access rows from `inputs` and `targets` as tuples. We'll also create a **DataLoader**, to split the data into batches while training. It also lets us shuffle and sample the data.

In [33]:
# Import tensor dataset and data loader
from torch.utils.data import TensorDataset, DataLoader

In [34]:
# Define Dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [35]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
next(iter(train_dl))

[tensor([[ 87., 134.,  58.],
         [102.,  43.,  37.],
         [ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [102.,  43.,  37.]]), tensor([[119., 133.],
         [ 22.,  37.],
         [ 56.,  70.],
         [ 81., 101.],
         [ 22.,  37.]])]

## SimpleNet

In [92]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3, 3)
        # Activation function
        self.act1 = nn.ReLU()
        self.linear2 = nn.Linear(3, 2)
    
    def forward(self, x):
        x = self.linear1(x)
        #x = self.act1(x)
        x = self.linear2(x)
        return x

This time, we will use our custom Module `SimpleNet`.

In [109]:
#model = nn.Linear(in_features=3, out_features=2)
model = SimpleNet()

##  Optimizer

Instead of manually manipulating weights and biases using gradients, we can use the optimizer `optim.SGD`

In [110]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

## Loss function

Instead of manually defining a loss function, we can use the built-in loss function `mse_loss`

In [111]:
# Import nn.functional
import torch.nn.functional as F

In [112]:
loss_fn = F.mse_loss

In [113]:
# Calculate loss

#preds = model(inputs)
#loss = loss_fn(preds, targets)

loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8850.6875, grad_fn=<MseLossBackward>)


## Train the model

We can train the model now. Let's define a utility function `fit` to train the model for a given number of epochs

In [114]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb, yb in train_dl:
            # Predictions
            preds = model(xb)
            loss = loss_fn(preds, yb)
            
            # Gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(inputs), targets))

In [118]:
# Train the model for 100 epochs
fit(num_epochs=100, model=model, loss_fn=loss_fn, opt=opt)

Training loss:  tensor(1.3141, grad_fn=<MseLossBackward>)


In [119]:
preds = model(inputs)
print(preds)
print(targets)

tensor([[ 57.1385,  70.4149],
        [ 82.9016, 100.0480],
        [117.3541, 134.5494],
        [ 21.0358,  37.1689],
        [103.1016, 117.9646],
        [ 57.1385,  70.4149],
        [ 82.9016, 100.0480],
        [117.3541, 134.5494],
        [ 21.0358,  37.1689],
        [103.1016, 117.9646],
        [ 57.1385,  70.4149],
        [ 82.9016, 100.0480],
        [117.3541, 134.5494],
        [ 21.0358,  37.1689],
        [103.1016, 117.9646]], grad_fn=<AddmmBackward>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])
